In [1]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR, MODELS_DIR
from omegaconf import OmegaConf
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from src.mimolm import InputProjections, EarlyFusionEncoder, MotionDecoder
import torch
import torch.nn.functional as F
import lightning as pl

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

2025-02-10 13:48:33.050 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/harshavardhan-patil/Work/Projects/mimolm


/home/harshavardhan-patil/Work/Projects/mimolm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/harshavardhan-patil/Work/Projects/mimolm/notebooks


In [4]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2
from src.mimolm import MimoLM
from lightning.pytorch.callbacks import LearningRateMonitor
from src.modeling.modules.lm_utils import interpolate_trajectory, cluster_rollouts, non_maximum_suppression

torch.set_printoptions(sci_mode=False)
data_module = DataH5av2(DATA_DIR / "interim" / "train")
data_module.setup(stage="validate")
val_loader = data_module.val_dataloader()

model = MimoLM.load_from_checkpoint(checkpoint_path=str(MODELS_DIR / "v3e3.ckpt" )
                                    , data_size=data_module.tensor_size_val
                                    , n_rollouts = 128
                                    , learning_rate = 8.e-8)
trainer = pl.Trainer(fast_dev_run=1)
output = trainer.test(model=model, dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1/1 [00:02<00:00,  0.45it/s]


In [5]:
model.validation_output

{'BrierMinFDE': array([57.0788338,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]),
 'MinFDE': array([57.0788338,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]),
 'MinADE': array([33.34519174,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ])}